In [ ]:
from src.utils.mics import get_all_gcd_files
from pathlib import Path
import numpy as np


def meshing_uv_map(occupancy):
    occ = occupancy.astype(bool)
    pixel_index = np.arange(occ.size).reshape(occ.shape)

    # Determine triangles' vertices
    is_tri_vert = (
        occ & np.roll(occ, shift=-1, axis=0) & np.roll(occ, shift=-1, axis=1)
    )
    verta = pixel_index
    vertb = np.roll(pixel_index, shift=-1, axis=1)
    vertc = np.roll(pixel_index, shift=-1, axis=0)
    face0 = np.stack(
        [verta[is_tri_vert], vertb[is_tri_vert], vertc[is_tri_vert]], axis=1
    )

    # Determine the second set of triangles' vertices
    is_tri_vert = (
        occ & np.roll(occ, shift=1, axis=0) & np.roll(occ, shift=1, axis=1)
    )
    verta = pixel_index
    vertb = np.roll(pixel_index, shift=1, axis=1)
    vertc = np.roll(pixel_index, shift=1, axis=0)
    face1 = np.stack(
        [verta[is_tri_vert], vertb[is_tri_vert], vertc[is_tri_vert]], axis=1
    )

    # Combine the two sets of faces
    face = np.concatenate([face0, face1], axis=0)

    return face


all_files = get_all_gcd_files()
for fi in all_files:
    omg_path = Path(fi) / f"{Path(fi).stem}_gim.npz"
    try:
        omg = np.load(omg_path)['arr_0']
    except:
        continue
    occupancy = np.any(omg > 0, axis=-1)
    faces = meshing_uv_map(occupancy)

    a = np.sum(occupancy.astype(int))
    b = np.unique(faces.flatten()).shape[0]
    if a != b:
        print(a, b, a - b, fi)
        # break


def create_obj(omg: np.ndarray, faces: np.ndarray):
    omg_size = omg.shape[0]

    idx = np.unique(faces.flatten())
    new_idx = {
        old: new + 1 for new, old in enumerate(idx)
    }
    list_vert = []
    for i in idx:
        list_vert.append(omg[(i // omg_size), (i % omg_size)])

    with open("output.obj", "w") as f:
        for vert in list_vert:
            f.write(f"v {vert[0]} {vert[1]} {vert[2]}\n")

        for face in faces:
            f.write(f"f {new_idx[face[0]]} {new_idx[face[1]]} {new_idx[face[2]]}\n")


# create_obj(omg, faces)

In [ ]:
# 111 points
# omg_path = "/home/phphuc/Desktop/GarmentRecon/GarmentCode/data/GarmentCodeData_v2/GarmentCodeData_v2/garments_5000_0/default_body/data/rand_H1HW5UW5S7/rand_H1HW5UW5S7_gim.npz"

omg_path = "/home/phphuc/Desktop/GarmentRecon/GarmentCode/data/GarmentCodeData_v2/GarmentCodeData_v2/garments_5000_0/default_body/data/rand_2E7LYE4R51/rand_2E7LYE4R51_gim.npz"
omg = np.load(omg_path)['arr_0']

occupancy = np.any(omg > 0, axis=-1)
faces = meshing_uv_map(occupancy)
create_obj(omg, faces)

In [ ]:
np.unique(faces.flatten()).shape

In [ ]:
tensor2omg(omg_tensor)

In [ ]:
from huggingface_hub import snapshot_download

# Define the model repository
model_name = "black-forest-labs/FLUX.1-dev-onnx"

# Download the model
local_model_path = snapshot_download(repo_id=model_name)

print(f"Model downloaded to: {local_model_path}")

In [ ]:
from transformers import AutoTokenizer
import onnxruntime as ort
import numpy as np
import torch

# Load tokenizer
model_name = "black-forest-labs/FLUX.1-dev-onnx"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load ONNX runtime session
onnx_model_path = f"{model_name}/model.onnx"
session = ort.InferenceSession(onnx_model_path)

# Example text input
text = "This is a test input."

# Tokenize input
inputs = tokenizer(text, return_tensors="np")

# Prepare inputs for ONNX
onnx_inputs = {k: v.astype(np.int64) for k, v in inputs.items()}

# Run inference
outputs = session.run(None, onnx_inputs)

# Print results
print(outputs)